In [71]:
import os
import pandas
import tqdm
import functools
from sklearn import preprocessing

In [72]:
source = '../resource/kaggle/sample/csv'
table = {
    'article':pandas.read_csv(os.path.join(source, 'articles.csv'), dtype=str),
    'customer':pandas.read_csv(os.path.join(source, 'customers.csv'), dtype=str),
    'transaction':pandas.read_csv(os.path.join(source, 'transactions_train.csv'), dtype=str),
    'submission':pandas.read_csv(os.path.join(source, 'sample_submission.csv'), dtype=str)
}

Fill the missing value.

In [73]:
table['article']['detail_desc'].fillna('miss_detail_desc', inplace=True)
table['customer']['FN'].fillna("0.0", inplace=True)
table['customer']['Active'].fillna("0.0", inplace=True)
table['customer']['fashion_news_frequency'].fillna("NONE", inplace=True)
table['customer']['age'].fillna("-1", inplace=True)
table['customer']['postal_code'].fillna('miss_postal_code', inplace=True)

Label encode the category variable, normalization of numerical variable.

Claim the column.

In [74]:
token = {"<padding>":0, "<start>":1}
claim = {
    'customer_id':'customer_id', 
    'length':'length', 
    'FN':"p1", 
    'Active':'p2', 
    'club_member_status':"p3",
    'fashion_news_frequency':'p4', 
    'age':'p5', 
    'postal_code':'p6', 
    'product_code':'b1', 
    'prod_name':'b2', 
    'product_type_no':'b3',
    'product_type_name':'b4', 
    'product_group_name':'b5', 
    'graphical_appearance_no':'b6',
    'graphical_appearance_name':"b7", 
    'colour_group_code':"b8",
    'colour_group_name':"b9",
    'perceived_colour_value_id':'b10', 
    'perceived_colour_value_name':"b11",
    'perceived_colour_master_id':"b12", 
    'perceived_colour_master_name':"b13",
    'department_no':'b14', 
    'department_name':"b15", 
    'index_code':"b16", 
    'index_name':"b17",
    'index_group_no':"b18", 
    'index_group_name':"b19", 
    'section_no':"b20", 
    'section_name':"b21",
    'garment_group_no':"b22", 
    'garment_group_name':"b23", 
    'detail_desc':"b24",
    'article_label':"b25",
    'price':'r1',
    'sales_channel_id':'r2', 
}

In [75]:
table['article']["article_label"] = table['article']['article_id']
for key in table['article']:

    if(key=='article_id'): continue

    skip = len(token)
    engine = preprocessing.LabelEncoder()
    engine.fit(table['article'][key])
    value = engine.transform(table['article'][key])
    table['article'][key] = value + skip
    continue

In [76]:
table['customer']['FN'] = table['customer']['FN'].astype(float)
table['customer']['Active'] = table['customer']['Active'].astype(float)
table['customer']['age'] = table['customer']['age'].astype(float) / 100
for key in table['customer']:

    if(key in ['customer_id', "FN", "Active", 'age']): continue

    skip = 0
    engine = preprocessing.LabelEncoder()
    engine.fit(table['customer'][key])
    value = engine.transform(table['customer'][key])
    table['customer'][key] = value + skip
    continue

In [77]:
table['transaction']['t_dat'] = pandas.to_datetime(table['transaction']['t_dat'])
value = table['transaction']['price'].astype(float).copy()
value = value / value.max()
table['transaction']['price'] = value
for key in table['transaction']:
    
    if(key in ['t_dat', "customer_id", "article_id", 'price']): continue

    skip = len(token)
    engine = preprocessing.LabelEncoder()
    engine.fit(table['transaction'][key])
    value = engine.transform(table['transaction'][key])
    table['transaction'][key] = value + skip
    continue

In [78]:
for column in table['customer']: 
    
    if(column in ["customer_id"]): continue

    max = round(table['customer'][column].max(), 3)
    min = round(table['customer'][column].min(), 3)
    unique = table['customer'][column].nunique()
    print("{} => \033[95m{}\033[00m | max:{} min:{} unique:{}".format(column, claim[column], max, min, unique))
    pass

FN => p1 | max:1.0 min:0.0 unique:2
Active => p2 | max:1.0 min:0.0 unique:2
club_member_status => p3 | max:3 min:0 unique:4
fashion_news_frequency => p4 | max:2 min:0 unique:3
age => p5 | max:0.98 min:-0.01 unique:78
postal_code => p6 | max:88828 min:0 unique:88829


In [79]:
for column in table['article']: 
    
    if(column in ["article_id"]): continue
    max = round(table['article'][column].max(), 3)
    min = round(table['article'][column].min(), 3)
    unique = table['article'][column].nunique()
    print("{} => \033[95m{}\033[00m | max:{} min:{} unique:{}".format(column, claim[column], max, min, unique))
    pass

product_code => b1 | max:13038 min:2 unique:13037
prod_name => b2 | max:13228 min:2 unique:13227
product_type_no => b3 | max:101 min:2 unique:100
product_type_name => b4 | max:100 min:2 unique:99
product_group_name => b5 | max:16 min:2 unique:15
graphical_appearance_no => b6 | max:30 min:2 unique:29
graphical_appearance_name => b7 | max:30 min:2 unique:29
colour_group_code => b8 | max:51 min:2 unique:50
colour_group_name => b9 | max:51 min:2 unique:50
perceived_colour_value_id => b10 | max:9 min:2 unique:8
perceived_colour_value_name => b11 | max:9 min:2 unique:8
perceived_colour_master_id => b12 | max:20 min:2 unique:19
perceived_colour_master_name => b13 | max:20 min:2 unique:19
department_no => b14 | max:276 min:2 unique:275
department_name => b15 | max:229 min:2 unique:228
index_code => b16 | max:11 min:2 unique:10
index_name => b17 | max:11 min:2 unique:10
index_group_no => b18 | max:6 min:2 unique:5
index_group_name => b19 | max:6 min:2 unique:5
section_no => b20 | max:57 min:2 u

In [80]:
for column in table['transaction']: 
    
    if(column in ['t_dat', 'customer_id', 'article_id']): continue

    max = round(table['transaction'][column].max(),3)
    min = round(table['transaction'][column].min(),3)
    unique = table['transaction'][column].nunique()
    print("\033[95m{}\033[00m => \033[91m{}\033[00m | max:{} min:{} unique:{}".format(column, claim[column], max, min, unique))
    continue

price => r1 | max:1.0 min:0.001 unique:2019
sales_channel_id => r2 | max:3 min:2 unique:2


Feature with personality and behavior.

In [81]:
paste = lambda x: " ".join(x)

In [82]:
sequence = []

In [83]:
table['transaction']['length'] = table['transaction']['customer_id']
sequence += [table['transaction'].groupby('customer_id').count().reset_index()[['customer_id', 'length']]]

In [84]:
table['transaction']['price'] = table['transaction']['price'].astype(str)
table['transaction']['sales_channel_id'] = table['transaction']['sales_channel_id'].astype(str)
sequence += [table['transaction'].groupby(['customer_id', 't_dat'])['price'].apply(paste).reset_index().groupby(['customer_id'])['price'].apply(paste).reset_index()]
sequence += [table['transaction'].groupby(['customer_id', 't_dat'])['sales_channel_id'].apply(paste).reset_index().groupby(['customer_id'])['sales_channel_id'].apply(paste).reset_index()]

In [85]:
for key in tqdm.tqdm(table['article'].keys()):

    if(key=='article_id'): continue

    table['article'][key] = table['article'][key].astype(str)
    selection = table['transaction'][['t_dat', "customer_id", "article_id"]].copy()
    selection = pandas.merge(selection, table['article'][["article_id", key]], on="article_id", how='inner')
    sequence += [selection.groupby(['customer_id', 't_dat'])[key].apply(paste).reset_index().groupby(['customer_id'])[key].apply(paste).reset_index()]
    continue

100%|██████████| 26/26 [02:09<00:00,  4.99s/it]


In [86]:
merge = lambda x,y: pandas.merge(left=x, right=y, on='customer_id', how='inner')
sequence = functools.reduce(merge, sequence)
sequence.head(2)

,customer_id,length,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,article_label
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,7,0.11458452722063038 0.0429512893982808 0.05727...,3 3 3 3 3 3 3,8731 4290 8654 1585 11105 11585 4290,2035 8441 9450 11130 5767 5288 8441,30 22 16 9 9 9 22,68 23 66 77 77 77 23,6 5 7 7 7 7 5,18 25 12 18 18 18 25,...,5 5 2 5 2 2 5,6 6 8 6 8 8 6,3 3 2 3 2 2 3,3 3 4 3 4 4 3,36 36 4 36 4 2 36,14 14 46 14 46 54 14,12 13 10 4 4 4 13,14 5 3 9 9 9 5,7841 8513 7670 4448 5514 4393 8513,17865 9584 17725 3974 21745 22544 9584
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,3,0.08593123209169053 0.051547277936962746 0.085...,3 3 3,10851 2075 10851,787 9744 787,33 52 33,90 13 90,14 14 14,18 18 18,...,3 3 3,9 9 9,2 2 2,4 4 4,43 43 43,48 48 48,16 16 16,20 20 20,4821 4835 4821,21308 4947 21308


In [87]:
table['feature'] = pandas.merge(left=table['customer'], right=sequence, on='customer_id', how='outer')

In [88]:
table['feature']['length'] = [str(int(i)) if(not pandas.isna(i)) else i for i in table['feature']['length']]

In [89]:
table['feature'] = table['feature'].rename(columns=claim)
table['feature'] = table['feature'][claim.values()]
table['feature'].head(2)

,customer_id,length,p1,p2,p3,p4,p5,p6,b1,b2,...,b18,b19,b20,b21,b22,b23,b24,b25,r1,r2
0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,7,0.0,0.0,0,1,0.25,14294,8731 4290 8654 1585 11105 11585 4290,2035 8441 9450 11130 5767 5288 8441,...,3 3 2 3 2 2 3,3 3 4 3 4 4 3,36 36 4 36 4 2 36,14 14 46 14 46 54 14,12 13 10 4 4 4 13,14 5 3 9 9 9 5,7841 8513 7670 4448 5514 4393 8513,17865 9584 17725 3974 21745 22544 9584,0.11458452722063038 0.0429512893982808 0.05727...,3 3 3 3 3 3 3
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,3,0.0,0.0,0,1,0.24,34910,10851 2075 10851,787 9744 787,...,2 2 2,4 4 4,43 43 43,48 48 48,16 16 16,20 20 20,4821 4835 4821,21308 4947 21308,0.08593123209169053 0.051547277936962746 0.085...,3 3 3


In [90]:
for origin, column in claim.items():

    message = "\033[95m{}\033[00m => \033[91m{}\033[00m".format(origin, column)
    print(message)
    continue

customer_id => customer_id
length => length
FN => p1
Active => p2
club_member_status => p3
fashion_news_frequency => p4
age => p5
postal_code => p6
product_code => b1
prod_name => b2
product_type_no => b3
product_type_name => b4
product_group_name => b5
graphical_appearance_no => b6
graphical_appearance_name => b7
colour_group_code => b8
colour_group_name => b9
perceived_colour_value_id => b10
perceived_colour_value_name => b11
perceived_colour_master_id => b12
perceived_colour_master_name => b13
department_no => b14
department_name => b15
index_code => b16
index_name => b17
index_group_no => b18
index_group_name => b19
section_no => b20
section_name => b21
garment_group_no => b22
garment_group_name => b23
detail_desc => b24
article_label => b25
price => r1
sales_channel_id => r2


In [91]:
folder = '../resource/preprocess/sample/csv/'
os.makedirs(folder, exist_ok=True)
table['feature'].dropna().to_csv(os.path.join(folder, "feature(train).csv"), index=False)
table['feature'].dropna().head(100).to_csv(os.path.join(folder, "feature(sample).csv"), index=False)
table['feature'].fillna("").to_csv(os.path.join(folder, "feature(all).csv"), index=False)

In [92]:
print(table['feature'].shape)
print(table['feature'].columns.tolist())

(114870, 35)
['customer_id', 'length', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15', 'b16', 'b17', 'b18', 'b19', 'b20', 'b21', 'b22', 'b23', 'b24', 'b25', 'r1', 'r2']


---

In [ ]:
    # if(column in ['customer_id', 'length']): 
        
    #     continue
    
    # if('p' in column): 

    #     max = table['feature'][column].max()
    #     min = table['feature'][column].min()
    #     unique = table['feature'][column].nunique()
    #     message = "[{}/{}] max:{} min:{} \033[95munique:{}\033[00m".format(origin, column, max, min, unique)
    #     print(message)
    #     continue

    # if('b' in column):

    #     value = sorted(set([float(j) for i in table['feature'][column].dropna() for j in i.split()]))
    #     min = int(value[0])
    #     max = int(value[-1])
    #     unique = len(value)
    #     message = "[{}/{}] \033[95mmax:{}\033[00m min:{} unique:{}".format(origin, column, max, min, unique)
    #     print(message)
    #     continue